### Cat or Dog?
#### CPSC 323-01
#### Sam Berkson
In this project, I will endeavor to create a convolutional neural network to classify images of cats and dogs.

##### Library Imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import os
import cv2 as ocv
import random
from datetime import datetime
import PIL
import pathlib
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

2023-04-01 13:51:41.049299: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


TypeError: Descriptors cannot not be created directly.
If this call came from a _pb2.py file, your generated code is out of date and must be regenerated with protoc >= 3.19.0.
If you cannot immediately regenerate your protos, some other possible workarounds are:
 1. Downgrade the protobuf package to 3.20.x or lower.
 2. Set PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION=python (but this will use pure-Python parsing and will be much slower).

More information: https://developers.google.com/protocol-buffers/docs/news/2022-05-06#python-updates

##### Data Loading and Proccessing
Here, I load all pictures of cats and dogs into two lists, resulting in 21896 total pictures.

In [ ]:
# Data load
data_dir = pathlib.Path('/Users/sberkson/Desktop/Gonzaga/JuniorYear/CPSC323-01/CatDogClassifier/PetImages')
dogs = list(data_dir.glob('Dog/*'))
cats = list(data_dir.glob('Cat/*'))
image_count = len(list(data_dir.glob('*/*.jpg')))

: 

In [ ]:
# Print first cat image 
PIL.Image.open(str(cats[4]))

: 

In [ ]:
# Print first dog image
PIL.Image.open(str(dogs[0]))

: 

Next, I build parallel dictionaries with the images and their binary classification labels.

In [ ]:
pet_images_dict = {'cat': list(cats), 'dog': list(dogs)}
pet_labels_dict = {'cat': 0, 'dog': 1}

: 

Next, I need to split images into X and Y while resizing images to a workable scale.  I scale the images to 128x128, add the resized images to X and their labels to Y, and then convert X and Y to numpy arrays.

In [ ]:
IMAGE_WIDTH=128
IMAGE_HEIGHT=128
X, Y = [], []

for pet_name, images in pet_images_dict.items():
    for image in images:
        img = ocv.imread(str(image))
        if isinstance(img,type(None)): 
            #print('image not found')
            continue
            
        elif ((img.shape[0] >= IMAGE_HEIGHT) and  (img.shape[1] >=IMAGE_WIDTH)):
            resized_img = ocv.resize(img,(IMAGE_WIDTH,IMAGE_HEIGHT))
            X.append(resized_img)
            Y.append(pet_labels_dict[pet_name])
        else:
            #print("Invalid Image")
            continue

X = np.array(X)
Y = np.array(Y)

: 

Next, I split my data into a train and test set.

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=.25 ,random_state=0)

: 

Checking my set shapes, it looks like everything came out in the dimensions I needed.  

In [ ]:
# Check shapes
print('X_train shape: ', X_train.shape)
print('y_train shape: ', Y_train.shape)
print('X_test shape: ', X_test.shape)
print('y_test shape: ', Y_test.shape)

: 

##### Classification
For my model, I used a convolutional neural network.  It goes through 3 iterations of:
* 2x Conv2D layers
* Batch Normalization
* MaxPool2D
* Dropout
After this, it spits into a feed-forward network and ending at our prediction of cat or dog.

In [ ]:
from keras.utils.vis_utils import plot_model

IMAGE_CHANNELS = 3

model = tf.keras.models.Sequential([
    # Convolutional network
    tf.keras.layers.Conv2D(32, (5, 5), activation='relu', input_shape=(IMAGE_WIDTH, IMAGE_HEIGHT, IMAGE_CHANNELS)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPool2D(pool_size=(2, 2)),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPool2D(pool_size=(2,2)),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPool2D(pool_size=(2,2)),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Flatten(),
    # Fully connected feed-forward network
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation='sigmoid') # Sigmoid for multi-class classification
])

plot_model(model, to_file='model.png', show_shapes=True, show_layer_names=True)

: 

Here I build my checkpoint system to implement early stopping after the model stops improving (after 5 epochs).

In [ ]:
# Build epoch checkpoint callback
callbacks = [
    tf.keras.callbacks.EarlyStopping(
        monitor = 'val_accuracy',
        patience = 5,
        verbose = 1,
        min_delta = 0,
        mode = 'max',
        baseline = None,
        restore_best_weights = True
    ),
    tf.keras.callbacks.ModelCheckpoint(
        os.path.join("/Users/sberkson/Desktop/Gonzaga/JuniorYear/CPSC323-01/CatDogClassifier", 'ckpt', "{epoch:02d}-{val_loss:.2f}.hdf5"),
        monitor = 'val_accuracy',
        verbose = 1,
        save_best_only = True,
        save_weights_only = False,
        mode = 'max',
        save_freq = 'epoch',
        options = None,
        initial_value_threshold = None
    )   
]

: 

Now its time to compile the model and train it.

In [ ]:
model.compile(optimizer = 'adam',
              loss = 'binary_crossentropy', 
              metrics = ['accuracy'])


start = datetime.time()
# Train the model
history = model.fit(X_train, Y_train, epochs = 500, 
                                      validation_split = 0.2, 
                                      callbacks = callbacks)
end = datetime.time()

print("Training time: ", end - start)

: 

##### Results
After 18 epochs, our model failed to improve over 5 epochs and early stopping was executed.  Lets run our test set and look at our final results.

In [ ]:
# Predict y values for the test set
start = datetime.time()
y_pred = model.predict(X_test)
end = datetime.time()
print("Prediction time: ", end - start)

: 

In [ ]:
# Output number of correct predictions
correct = 0

for i in range(len(y_pred)):
    if y_pred[i] >= .5:
        y_pred[i] = 1
    else:
        y_pred[i] = 0
    if y_pred[i] == Y_test[i]:
        correct += 1
print('Correct predictions: ', correct)
print('Incorrect predictions: ', len(y_pred) - correct)
print('Accuracy: ', correct / len(y_pred))

: 

In [ ]:
# Load IMG_7840.jpeg into a separate variable to test individually
img = ocv.imread('IMG_7840.jpeg')
resized_img = ocv.resize(img,(IMAGE_WIDTH,IMAGE_HEIGHT))
cleo = np.array(resized_img)

: 

82% accuracy isnt bad, particularly considered I downscaled my initial version of the neural network to improve run speed (15 minutes per epoch before!).  Looks like simplicity was the way to go here.  Lets visualize our results now.

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns

# Create confusion matrix
cm = confusion_matrix(Y_test, y_pred)

# Plot confusion matrix
plt.figure(figsize = (10, 7))
sns.heatmap(cm, annot = True, fmt = 'd')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

: 

In [ ]:
# Plot training & validation accuracy values
plt.figure()
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

: 

It seems my model was ~ok~ at self correcting, although it took 2 or 3 epochs to get itself back on the right track in some of the dips.  Overall, im pretty happy with the results.  Thanks for a great semester!

In [ ]:
# Checking if my cat is a cat
y_pred = model.predict(cleo.reshape(1, 128, 128, 3))
print(y_pred)

: 

In [ ]:
# Its a dog, knew it

: 